In [12]:
import numpy as np
import pandas as pd

df = pd.read_csv('C:/Users/Orestis/Desktop/1.Diplomatiki/Data/Item Data/imagenewsreel-itemData.csv', delimiter=';', header=None, escapechar='\\', quotechar='\"')
df.columns = ['id','imageId','title','publisher','url','image_url','stemmed','date','article_id']

In [13]:
#sbinw tis NaN
nonull_df = df.dropna(how='any', axis = 0)
nonull_df.shape

(51368, 9)

In [14]:
final_df = nonull_df.drop_duplicates(subset = 'article_id')
final_df.shape,type(final_df)

((51260, 9), pandas.core.frame.DataFrame)

In [ ]:
final_df

In [15]:
image_url = final_df.loc[: , [5,8]]
image_url
image_url.to_csv("C:/Users/Orestis/Desktop/1.Diplomatiki/Data/imageUrls", index= False , header = None)

KeyError: 'None of [[5, 8]] are in the [columns]'

In [ ]:
date = pd.to_datetime(df[7])
date

In [ ]:
stemmed = []
stemmed = final_df['stemmed']
stemmed.shape
stemmed

In [19]:
# poios einai o megistos arithmos leksewn stous titlous
max_words = 0
for x in stemmed:
    s = x.split(";")
    counter = 0
    for word in s:
        counter+=1
    if counter>max_words:
        max_words = counter
print(max_words)

30


In [20]:
voc2={} #poses fores emfanizetai h kathe leksi
for x in stemmed:
    s = x.split(";")
    for word in s:
        if word not in voc2:
            voc2[word] = 1
        else: 
            voc2[word] = voc2[word] + 1
        
len(voc2)


80375

In [21]:
import operator #most used lekseis
sorted_voc = sorted(voc2.items(), key=operator.itemgetter(1), reverse = True)
sorted_voc
most_used = []
for x in range(0,20000):
    most_used.append(sorted_voc[x][0])
set(most_used) # to metatrepw apo list se set{} 
most_used = set(most_used)
len(most_used)


20000

In [22]:
#dimiourgw vocabulary(dictionary) me oles tis lekseis kai tous bazw enan auksanwn arithmo diaforetiko gia kathe leksi
counter = 0  
vocabulary={}
for x in stemmed:
    s = x.split(";")
    for word in s:
        if (word not in vocabulary) and (word in most_used):   #extra: theloume vocabulary twn most_used leksewn
            #An kaname vocabulary gia oles tis lekseis tha paraleipame to end
            vocabulary[word] = counter
            counter+=1
len(vocabulary), max(vocabulary.values())

(20000, 19999)

In [24]:
import pickle
with open("my_vocabulary.pk", 'wb') as handle:
    pickle.dump(vocabulary, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [71]:
word_codes = []   # lista me tous arithmous twn pio xrisimopoihmenwn leksewn tou kathe article
for x in stemmed:
    numbers = []
    s = x.split(";")
    for word in s:
        if word in vocabulary:  #ean i leksi einai stis most_used
            numbers.append(vocabulary[word])
   # if len(numbers) < 30:
    for i in range(30 - len(numbers)):
        numbers.append(20000)            ##prosthetw ton kwdiko 20000 gia ton <PAD> xaraktira wste na exoun oloi oi titloi to idio mikos
    word_codes.append(numbers)
len(word_codes[578])

30

In [72]:
type(word_codes)
#word_codes = np.array(word_codes)
#word_codes.shape
#word_codes[0:6]

list

In [73]:
counter = 0 #posa arthra den periexoun kamia apo tis suxnoteres 20k lekseis
for x in word_codes:
    if len(x)== 0:
        counter+=1
print(counter)
    

0


In [74]:
#prosthetw tous kwdikous twn leksewn sto dataframe
df[9]= word_codes
df[9]

0        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 20000, ...
1        [12, 13, 14, 15, 16, 17, 18, 19, 20000, 20000,...
2        [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 2...
3        [31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 4...
4        [43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 5...
5        [41, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 6...
6        [77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 8...
7        [89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 20000...
8        [0, 99, 100, 101, 102, 103, 104, 8, 20000, 200...
9        [105, 105, 106, 107, 108, 109, 110, 105, 20000...
10       [111, 112, 113, 114, 115, 105, 3, 20000, 20000...
11       [116, 117, 7, 118, 119, 120, 121, 122, 123, 7,...
12       [127, 128, 3, 129, 130, 131, 20000, 20000, 200...
13       [132, 71, 133, 134, 135, 136, 10, 137, 138, 13...
14       [7, 143, 144, 145, 35, 146, 147, 148, 149, 150...
15       [105, 151, 78, 152, 153, 110, 154, 155, 126, 1...
16       [159, 3, 160, 161, 20000, 20000, 20000, 20000,.

In [75]:
#elegxoume ean ontws den ksefeugei panw apo ton arithmo 20000
max_num = 0
for x in df[9]:   #to x einai lista
    if (len(x)!=0) and (max(x) > max_num):
        max_num = max(x)
print(max_num)
    

20000


In [54]:
#ftiaxnw enan one hot vector gia kathe arthro.Rows: oi lekseis kathe arthrou, Columns: Oles oi lekseis tou voc. Tha prepei na exoume 1 opou uparxei h leksi
#sto arhtro kai 0 se oles tis upoloipes lekseis.
def one_hot_vector_transformation(words):
    word_array = np.zeros([len(words),len(vocabulary)+1])
    for index,word in enumerate(words):
        word_array[index][word] = 1  #epistrefei numpy array
    return word_array

#H lista one_hot_vectors[] periexei pinakes numpy pou o kathenas antistoixei ston one_hot_vector kathe arthrou
one_hot_vectors = []
for words in df[9][:10]:
    one_hot_vectors.append(one_hot_vector_transformation(words))
    # one_hot_vectors += [one_hot_vector_transformation(words)]
    print(one_hot_vectors[-1].shape, one_hot_vectors[-1]) 
     #to one_hot_vectors einai lista epomenws dn exei shape()..omws me to -1 emfanizoume
    # mono tin teleutaia eggrafi i opoia einai ena numpy array to opoio einai auto pou epistrefei i sunartisi poy eftiaksa


(30, 20001) [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
(30, 20001) [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
(30, 20001) [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
(30, 20001) [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
(30, 20001) [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
(30, 20001) [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 

In [84]:
#o pinakas autos periexei random times pou tha pollaplasiastoun me tous one_hot_vectors
#H_embeddings = np.random.rand(len(vocabulary)+1,300)
#print(H_embeddings)
def one_hot_vector_transformation(words):
    word_array = np.zeros([len(words),len(vocabulary)+1])
    for index,word in enumerate(words):
        word_array[index][word] = 1  #epistrefei numpy array
    return word_array

def embedding_transformation(words):
    '''
    oi parakatw 3 grammes kanoyn to idio me tis apo katw 2
    '''
    vector = np.zeros((30, 300))
    for i, w in enumerate(words):
        vector[i] = H_embeddings[w]
        
    #vector = one_hot_vector_transformation(words)
    #vector = vector.dot(H_embeddings)
    return vector

# H lista embeddings[] periexei pinakes numpy pou o kathenas antistoixei sto apotelesma tou pollaplasiasmou tou H_embedding[] me 
# ton one_hot_vector kathe arthrou
embeddings = []
for words in df[9][:10]:
    embeddings.append(embedding_transformation(words))
    print(embeddings[-1].shape, embeddings[-1])

(30, 300) [[0.18135934 0.41970526 0.16698188 ... 0.53434933 0.05600806 0.53287283]
 [0.67450917 0.65482662 0.99785198 ... 0.80960158 0.63630658 0.24726745]
 [0.0634521  0.95788939 0.01054628 ... 0.83774346 0.85347967 0.84625822]
 ...
 [0.55391565 0.2155333  0.6243213  ... 0.23246399 0.56436811 0.84316905]
 [0.55391565 0.2155333  0.6243213  ... 0.23246399 0.56436811 0.84316905]
 [0.55391565 0.2155333  0.6243213  ... 0.23246399 0.56436811 0.84316905]]
(30, 300) [[0.47322562 0.33590667 0.78651973 ... 0.52065566 0.36745665 0.17580223]
 [0.58849603 0.81993792 0.99082775 ... 0.85088329 0.65788685 0.50516589]
 [0.56041381 0.74557034 0.41321277 ... 0.95519717 0.82012821 0.54461639]
 ...
 [0.55391565 0.2155333  0.6243213  ... 0.23246399 0.56436811 0.84316905]
 [0.55391565 0.2155333  0.6243213  ... 0.23246399 0.56436811 0.84316905]
 [0.55391565 0.2155333  0.6243213  ... 0.23246399 0.56436811 0.84316905]]
(30, 300) [[0.22750931 0.34779419 0.32545079 ... 0.49625177 0.29168358 0.85268501]
 [0.94465

In [82]:
#edw thelw na dimiourgisw ena pinaka pou tha exei ta average kathe stilis tis listas embeddings[].Ousiastika anti na exw ena pinaka me embeddings
#pou kathe grammh tha antistoixei se kathe leksi, tha exw enan monodiastato pinaka pou tha antistoixei se olh thn protash/titlo.
def embedding_transformation(words):
    vector = one_hot_vector_transformation(words)
    vector = vector.dot(H_embeddings)
    return vector

def average_vector(X):
    average = np.mean(X,axis=0)
    return average

#edw thelw na dimiourgisw ena pinaka pou tha exei ta average kathe stilis tis listas embeddings[].Ousiastika anti na exw ena pinaka me embeddings
#pou kathe grammh tha antistoixei se kathe leksi, tha exw enan monodiastato pinaka pou tha antistoixei se olh thn protash/titlo.
average_embeddings = []
for words in df[9][:10]:
    X = embedding_transformation(words) #epistrefei numpy array [words arthrou x 300]
    X = average_vector(X)               #epitrefei ena monodiastato pinaka me tis average times kathe leksis tou arhtrou
    print(X.shape)
    average_embeddings.append(X)        #apothikeuei ton monodiastato pinaka gia kathe arthro ksexwrista
    #print(average_embeddings[-1].shape, average_embeddings[-1])



(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)
(300,)


In [59]:
H_embeddings_2 = np.random.rand(300,150)
average_array = np.array(average_embeddings)  # metatrepw thn lista se numpy array gia na parw to .dot()

for rows in average_array:
    average_embeddings2 = average_array.dot(H_embeddings_2)
average_embeddings2.shape
    

(10, 150)

In [61]:
H_embeddings_3 = np.random.rand(150,1)
for rows in average_embeddings2:
    final_average = average_embeddings2.dot(H_embeddings_3)
final_average.shape, final_average

((10, 1), array([[6223.75865414],
        [6261.83218863],
        [6211.55453717],
        [6170.91443504],
        [6146.81914427],
        [6040.998535  ],
        [6184.37843125],
        [6228.9226018 ],
        [6273.17550502],
        [6323.23281855]]))

In [ ]:
####  ALLOS TROPOS-----NTEN NTOULEUEI
def h_embeddings_creation(col_num):
    row_num = col_num
    if col_num == 150:
        col_num = 1
        embedding_matrix = np.ranmdom.rand(row_num,col_num)
        return embedding_matrix
    else:
        embedding_matrix = np.random.rand(row_num,col_num/2)
        return embedding_matrix

average_array2 = np.array(average_embeddings)
end = 300
while end>1:
    for rows in average_array:
        final_average = average_array2.dot(h_embeddings_creation(end))
final_average.shape

In [ ]:
####### DEN XREIAZETAI TELIKA
data = np.array(embeddings)
print((data.shape))
data = data.reshape(data.shape[0],1)
print((data.shape),data)

In [62]:
########## CONVOLUTION_2D
from scipy import signal

#conv_embeddings = []
#for i in range(100):
#     conv_embeddings[i] = np.random.rand(2,300)
#conv_embeddings.shape

conv_embeddings = []
for i in range(100):
    m = np.random.rand(2,300)
    conv_embeddings.append(m)
    
conv_embeddings = np.array(conv_embeddings)       
#print(conv_embeddings.shape)

#an ithela na efarmosw enan pinaka conv_embedding tha ekana apla: 
# for article in enumerate(embeddings):
#        grad = signal.convolve2d(article, cv, boundary='symm', mode='valid')
# Twra omws tha efarmosw pollous pinakes conv_embeddings stis lekseis twn arthrwn opote gia kathe arthro tha exw polla apotelesmata grad. Ena
#gia kathe conv_embedding pou efarmozw, dld sto paradeigma 100...Ara 10 arthra x 100 dusdiastatous pinakes grad.
final_grad = []
for index,article in enumerate(embeddings):
    #final_grad.append([])
    temp = []
    print(article.shape)
    for cv in conv_embeddings:
    #print(article,article.shape)
        grad = signal.convolve2d(article, cv, boundary='symm', mode='valid')
        temp.append(grad)
    #cancatenate
    #temp = np.array(temp)
    temp = np.concatenate(temp, axis = 1)
    print(temp.shape)
    print("\n")
    final_grad.append(temp)
        #print(final_grad[-1]) xamos
#final_grad_array = np.array(final_grad)
print(len(final_grad),len(final_grad[1]))
#print(final_grad[1])

(30, 300)
(29, 100)


(30, 300)
(29, 100)


(30, 300)
(29, 100)


(30, 300)
(29, 100)


(30, 300)
(29, 100)


(30, 300)
(29, 100)


(30, 300)
(29, 100)


(30, 300)
(29, 100)


(30, 300)
(29, 100)


(30, 300)
(29, 100)


10 29


In [27]:
conv_embedding.shape

NameError: name 'conv_embedding' is not defined

In [ ]:
conv_embeddings = []
for i in range(100):
 conv_embeddings[i] = np.random.rand(2,300)
conv_embeddings.shape

In [ ]:
#### AKURO
word_num = []   # lista me tous arithmous twn leksewn tou kathe article me thn xrhsh tou genikou vocabulary, prepei na ftiaksw prwta to voc
for x in stemmed:
    numbers = []
    s = x.split(";")
    for word in s:
        numbers.append(vocabulary[word])
    word_num.append(numbers)
len(word_num)

In [4]:
data = df.as_matrix()
allpub = data[:, 3]
allpub.size

51397

In [5]:
unpub = []
for x in allpub:
    if x not in unpub:
        unpub.append(x)
print(unpub)

[17614, 1677, 39234, 13554, 694]


In [6]:
# alliws ginete kai etsi 
#uniquepub = np.unique(allpub) , dld me tin xrisi tis sunartisis np.unique()
set(unpub)

{694, 1677, 13554, 17614, 39234}

In [7]:
image = np.load('C:/Users/Orestis/Desktop/1.Diplomatiki/Data/Image Data/contestlog2018-01-06IMG/0.npy')

In [8]:
image.shape

(1, 14, 14, 512)

In [12]:
m = np.max(image , axis = (1,2))
m.shape

(1, 512)

In [22]:
df[1].max()-df[1].min(), df[1].shape

(51687, (51397,))